In [1]:
import os
import docx2txt
import PyPDF2
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
np.random.seed(0) 

In [2]:
directory = 'C:/Users/arund/OneDrive/Desktop/NLP/Project/Data/Resumes'
extension = '.docx'
text = ''
resume_dict = {}
for a_file in os.listdir(directory):
    content = []
    if a_file.endswith('.docx'):
        text = docx2txt.process(directory + '/' + str(a_file))
        #print(text, '\n\n------------------------------------------------------------------------------------------\n\n')
        resume_dict[str(a_file[:-5])] = text

In [3]:
resume_data = pd.DataFrame.from_dict(resume_dict, orient  = 'index', columns = ['Resume'])
resume_data.index.name = 'Names'
resume_data.head()

,Resume
Names,
ABHAY RAJ DWIVEDI,ABHAY RAJ DWIVEDIFeatured Resume \n\n \n\nSof...
Abhishek Sharma,Abhishek SharmaFeatured Resume \n\n \n\nDevel...
ADRYJA GHOSH,ADRYJA GHOSHFeatured Resume \n\n \n\nEnglish ...
AJAY CHINNI,AJAY CHINNIFeatured Resume \n\n \n\nSeeking r...
ALANKRIT NIRJHAR,ALANKRIT NIRJHARPremium CV Featured Resume \n...


In [4]:
directory = 'C:/Users/arund/OneDrive/Desktop/NLP/Project/Data/JobDesc'
extension = '.docx'
text = ''
job_desc_dict = {}
for a_file in os.listdir(directory):
    content = []
    if a_file.endswith('.docx'):
        text = docx2txt.process(directory + '/' + str(a_file))
        #print(text, '\n\n------------------------------------------------------------------------------------------\n\n')
        job_desc_dict[str(a_file[:-5])] = text

In [5]:
job_desc_data = pd.DataFrame.from_dict(job_desc_dict, orient  = 'index', columns = ['Job_Description'])
job_desc_data.index.name = 'Profile'
job_desc_data.head()

,Job_Description
Profile,
Backend Developer,Backend Developer - Python/Django\n\n\n\nWe ar...
Billing cum Logistics Manager,Billing cum Logistics Manager\n\nJob Summary\n...
Data Scientist,Data Scientist/ ML Engineer\n\nGame Change Sol...
Director of Engineering,"Director of Engineering, Google CloudDirector ..."
Global Industry Content Manager,Global Industry Content Manager\n\nMinimum qua...


## NLTK

In [6]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Cleaning Data

In [7]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import nltk.stem
from nltk.stem import LancasterStemmer, SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
from string import punctuation

In [8]:
'''Checks for punctuations and stopwords, converts to lower case, performs tokenization, POS tagging and lemmatization and 
also removes irrelevant data''' 

def clean_data(text): 
    stop_words = stopwords.words('english')
    tokens = word_tokenize(text)
    cleaned_tokens = []

    for tok, tag in pos_tag(tokens):
        tok = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\)]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', tok)
        tok = re.sub("(@[A-Za-z0-9_]+)","", tok)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        tok = lemmatizer.lemmatize(tok, pos)

        if len(tok) > 0 and tok not in punctuation and tok.lower() not in stop_words:
            cleaned_tokens.append(tok.lower())
    return cleaned_tokens

## Topic Modelling

In [9]:
doc = resume_data[['Resume']]
doc.sample(5)

,Resume
Names,
MADHU H,MADHU H.KPremium CV Featured Resume \n\n \n\n...
Manvi Singh,Manvi SinghFeatured Resume \n\n \n\nHR Manage...
MONICA CHOPRA,MONICA CHOPRAFeatured Resume \n\n \n\nForeign...
RISHABH SHARMA,RISHABH SHARMAFeatured Resume \n\n \n\nJunior...
Seshadri Sastry Kunapuli,Seshadri Sastry Kunapuli \n\n \n\nManager ...


In [10]:
import gensim
import re
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer, LancasterStemmer
from nltk.stem.porter import *
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from string import punctuation
np.random.seed(2018)

In [11]:
stemmer = SnowballStemmer('english')

In [12]:
processed_docs = doc['Resume'].map(clean_data)

In [13]:
processed_docs[:10]

Names
ABHAY RAJ DWIVEDI       [abhay, raj, dwivedifeatured, resume, software...
Abhishek Sharma         [abhishek, sharmafeatured, resume, developer, ...
ADRYJA GHOSH            [adryja, ghoshfeatured, resume, english, teach...
AJAY CHINNI             [ajay, chinnifeatured, resume, seeking, role, ...
ALANKRIT NIRJHAR        [alankrit, nirjharpremium, cv, featured, resum...
Amarpreet Singh         [amarpreet, singhpremium, cv, featured, resume...
AMITABHA                [amitabha, ghoshpremium, cv, featured, resume,...
Arun Kumar              [arun, kumar, ufeatured, resume, hr, manager, ...
Ashish Singh            [ashish, singh, thakurfeatured, resume, hr, ma...
Ashwani Kumar Rajput    [ashwani, kumar, rajputpremium, cv, featured, ...
Name: Resume, dtype: object

In [14]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [15]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 'india
1 's
2 'world
3 .net
4 1
5 1+
6 15
7 17-sep-20
8 17-sep-20last
9 2018
10 2019


## Bag of Words

In [16]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[10]

[(9, 5),
 (10, 2),
 (12, 2),
 (17, 1),
 (20, 1),
 (23, 3),
 (25, 22),
 (27, 1),
 (33, 1),
 (34, 2),
 (35, 6),
 (36, 1),
 (38, 1),
 (40, 3),
 (43, 1),
 (44, 4),
 (46, 2),
 (48, 1),
 (52, 2),
 (54, 4),
 (55, 3),
 (56, 2),
 (57, 1),
 (61, 1),
 (64, 2),
 (65, 1),
 (66, 1),
 (67, 3),
 (70, 1),
 (71, 7),
 (72, 7),
 (76, 3),
 (78, 1),
 (79, 2),
 (80, 3),
 (83, 3),
 (84, 1),
 (85, 1),
 (87, 1),
 (88, 6),
 (92, 3),
 (94, 1),
 (95, 2),
 (98, 1),
 (99, 1),
 (100, 1),
 (103, 2),
 (107, 1),
 (114, 1),
 (116, 6),
 (120, 2),
 (121, 1),
 (122, 1),
 (123, 1),
 (124, 1),
 (125, 1),
 (126, 1),
 (129, 6),
 (131, 2),
 (135, 4),
 (136, 13),
 (141, 1),
 (143, 1),
 (145, 1),
 (146, 2),
 (147, 3),
 (150, 1),
 (151, 3),
 (154, 1),
 (156, 1),
 (157, 3),
 (158, 6),
 (162, 1),
 (163, 1),
 (165, 5),
 (167, 1),
 (169, 1),
 (172, 1),
 (173, 12),
 (174, 2),
 (175, 2),
 (181, 3),
 (183, 1),
 (184, 4),
 (185, 1),
 (186, 2),
 (191, 1),
 (193, 1),
 (195, 2),
 (196, 4),
 (197, 1),
 (199, 1),
 (202, 1),
 (203, 3),
 (204, 10

In [17]:
bow_doc_10 = bow_corpus[10]

for i in range(len(bow_doc_10)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_10[i][0], 
                                                     dictionary[bow_doc_10[i][0]], 
                                                     bow_doc_10[i][1]))

Word 9 ("2018") appears 5 time.
Word 10 ("2019") appears 2 time.
Word 12 ("3") appears 2 time.
Word 17 ("active") appears 1 time.
Word 20 ("agile") appears 1 time.
Word 23 ("analysis") appears 3 time.
Word 25 ("application") appears 22 time.
Word 27 ("area") appears 1 time.
Word 33 ("aug") appears 1 time.
Word 34 ("b.tech/b.e") appears 2 time.
Word 35 ("back") appears 6 time.
Word 36 ("bangalore") appears 1 time.
Word 38 ("bengaluru") appears 1 time.
Word 40 ("business") appears 3 time.
Word 43 ("case") appears 1 time.
Word 44 ("client") appears 4 time.
Word 46 ("code") appears 2 time.
Word 48 ("company") appears 1 time.
Word 52 ("create") appears 2 time.
Word 54 ("css") appears 4 time.
Word 55 ("current") appears 3 time.
Word 56 ("data") appears 2 time.
Word 57 ("date") appears 1 time.
Word 61 ("degree") appears 1 time.
Word 64 ("design") appears 2 time.
Word 65 ("designation") appears 1 time.
Word 66 ("detail") appears 1 time.
Word 67 ("details") appears 3 time.
Word 70 ("developed")

## TF-IDF Vectorizer

In [18]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [19]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.05854746539431067),
 (1, 0.03210010617032598),
 (2, 0.05854746539431067),
 (3, 0.02737767107618461),
 (4, 0.010298131250104106),
 (5, 0.05854746539431067),
 (6, 0.013463642263235204),
 (7, 0.021109838395992522),
 (8, 0.01921556409829409),
 (9, 0.032337174434220455),
 (10, 0.017045137629016126),
 (11, 0.011792773917121576),
 (12, 0.02939596865833253),
 (13, 0.02904853942229823),
 (14, 0.05854746539431067),
 (15, 0.05854746539431067),
 (16, 0.035716003733171284),
 (18, 0.3007379776467335),
 (19, 0.05854746539431067),
 (20, 0.02737767107618461),
 (21, 0.07776302949260476),
 (22, 0.07776302949260476),
 (23, 0.032337174434220455),
 (24, 0.04906191930634081),
 (25, 0.0585843119882469),
 (26, 0.03888151474630238),
 (28, 0.16307632311067544),
 (29, 0.048714490070306524),
 (30, 0.024530959653170405),
 (31, 0.04296256823524764),
 (32, 0.09742898014061305),
 (33, 0.015357916560933637),
 (34, 0.017892124988214752),
 (35, 0.0009226860036190692),
 (36, 0.08025026542581494),
 (37, 0.0258830284

## LDA on Bag of Words Corpus

In [20]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [21]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {} \n'.format(idx, topic))

Topic: 0 
Words: 0.012*"software" + 0.011*"management" + 0.011*"experience" + 0.011*"project" + 0.009*"development" + 0.007*"top" + 0.007*"back" + 0.007*"year" + 0.007*"current" + 0.006*"skills" 

Topic: 1 
Words: 0.013*"management" + 0.010*"team" + 0.007*"experience" + 0.007*"marketing" + 0.007*"year" + 0.007*"business" + 0.006*"month" + 0.006*"•" + 0.005*"web" + 0.005*"development" 

Topic: 2 
Words: 0.021*"management" + 0.017*"employee" + 0.017*"hr" + 0.010*"experience" + 0.009*"manager" + 0.007*"business" + 0.006*"development" + 0.006*"year" + 0.005*"work" + 0.005*"sales" 

Topic: 3 
Words: 0.015*"sales" + 0.014*"management" + 0.013*"business" + 0.013*"marketing" + 0.008*"product" + 0.007*"experience" + 0.007*"channel" + 0.007*"market" + 0.006*"retail" + 0.006*"development" 

Topic: 4 
Words: 0.011*"student" + 0.010*"development" + 0.008*"management" + 0.008*"teacher" + 0.006*"english" + 0.006*"experience" + 0.005*"role" + 0.005*"education" + 0.005*"work" + 0.005*"learning" 

Topic

## LDA on TF-IDF Corpus

In [22]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [23]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {} \n'.format(idx, topic))

Topic: 0 Word: 0.001*"student" + 0.001*"trading" + 0.001*"price" + 0.001*"pgt" + 0.001*"educational" + 0.001*"exchange" + 0.001*"lucknow" + 0.001*"teacher" + 0.001*"school" + 0.001*"symphony" 

Topic: 1 Word: 0.002*"e" + 0.001*"student" + 0.001*"xamarin" + 0.001*"•" + 0.001*"linq" + 0.001*"asp.net" + 0.001*"teacher" + 0.001*"ef" + 0.001*"sr." + 0.001*"core" 

Topic: 2 Word: 0.002*"•" + 0.001*"hr" + 0.001*"employee" + 0.001*"student" + 0.001*"luxora" + 0.001*"pgt" + 0.001*"m.b.girls" + 0.001*"offsite" + 0.001*"learning" + 0.001*"teacher" 

Topic: 3 Word: 0.002*"sharepoint" + 0.001*"machine" + 0.001*"learning" + 0.001*"cloud" + 0.001*"mvc" + 0.001*"jquery" + 0.001*"developer" + 0.001*"spring" + 0.001*"4" + 0.001*"software" 

Topic: 4 Word: 0.003*"hr" + 0.003*"employee" + 0.001*"salary" + 0.001*"manager" + 0.001*"recruitment" + 0.001*"payroll" + 0.001*"attendance" + 0.001*"administration" + 0.001*"exit" + 0.001*"statutory" 

Topic: 5 Word: 0.002*"sales" + 0.002*"http" + 0.001*"c++" + 0.00

## LSA on Bag of Words Corpus

In [24]:
lsa_model = gensim.models.LsiModel(bow_corpus, num_topics=10, id2word=dictionary)

In [25]:
for idx, topic in lsa_model.print_topics(-1):
    print('Topic: {} Word: {} \n'.format(idx, topic))

Topic: 0 Word: 0.331*"management" + 0.226*"experience" + 0.223*"software" + 0.209*"project" + 0.184*"development" + 0.169*"employee" + 0.168*"application" + 0.168*"hr" + 0.167*"year" + 0.138*"developer" 

Topic: 1 Word: -0.366*"employee" + -0.342*"hr" + -0.297*"management" + 0.270*"software" + 0.202*"application" + 0.182*"project" + 0.178*"developer" + 0.168*"sharepoint" + -0.153*"manager" + -0.104*"recruitment" 

Topic: 2 Word: 0.930*"--" + 0.192*"english" + 0.086*"student" + 0.084*"teacher" + -0.083*"sharepoint" + -0.070*"hr" + -0.069*"employee" + 0.060*"teaching" + 0.055*"literature" + 0.050*"classroom" 

Topic: 3 Word: -0.299*"marketing" + -0.290*"sales" + 0.285*"hr" + 0.283*"employee" + -0.237*"business" + -0.198*"sale" + -0.179*"market" + -0.175*"channel" + -0.158*"team" + -0.148*"customer" 

Topic: 4 Word: 0.498*"sharepoint" + -0.199*"software" + 0.171*"2013" + 0.161*"2010" + -0.158*"•" + 0.151*"--" + 0.147*"site" + 0.136*"client" + -0.128*"experience" + 0.124*"project" 

Topic:

## LSA on TF-IDF Corpus

In [26]:
lsa_model_tfidf = gensim.models.LsiModel(corpus_tfidf, num_topics=10, id2word=dictionary)

In [27]:
for idx, topic in lsa_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {} \n'.format(idx, topic))

Topic: 0 Word: 0.317*"•" + 0.210*"employee" + 0.206*"hr" + 0.156*"software" + 0.132*"developer" + 0.126*"sales" + 0.120*"application" + 0.115*"asp.net" + 0.109*"web" + 0.101*"server" 

Topic: 1 Word: 0.376*"employee" + 0.368*"hr" + -0.236*"•" + -0.134*"software" + 0.130*"recruitment" + 0.123*"manager" + -0.116*"developer" + 0.112*"payroll" + -0.105*"asp.net" + 0.097*"sales" 

Topic: 2 Word: -0.424*"sales" + -0.309*"marketing" + 0.279*"•" + 0.195*"hr" + 0.186*"employee" + -0.177*"channel" + -0.154*"sale" + -0.133*"business" + -0.130*"market" + -0.129*"brand" 

Topic: 3 Word: 0.376*"machine" + -0.357*"•" + 0.325*"learning" + -0.159*"sales" + 0.155*"deep" + 0.149*"python" + 0.125*"model" + 0.121*"image" + -0.121*"marketing" + 0.121*"engineer" 

Topic: 4 Word: -0.615*"•" + 0.200*"asp.net" + -0.183*"machine" + -0.164*"learning" + 0.154*"software" + 0.141*"mvc" + 0.127*"developer" + 0.124*"jquery" + 0.102*"ajax" + 0.094*"javascript" 

Topic: 5 Word: -0.519*"student" + -0.354*"teacher" + -0.2